In [1]:
import pandas as pd
from decimal import Decimal
import math
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-4

In this demo we investigate the **Fast Router performance** against test orders

Initialize a fast simulator

In [2]:
Sim = CarbonSimulatorUI(pair="USDC/ETH", verbose=False, matching_method='exact', raiseonerror=True)
FastSim = CarbonSimulatorUI(pair="USDC/ETH", verbose=False, matching_method='fast', raiseonerror=True)

Here we restrain the orders to just the top ones such that we can test the exact algo alongside

In [3]:
test_orders = pd.read_csv('256 orders with curr rate alternating.csv', dtype=float)
test_orders.rename(columns={'lowest-rate':'lowest_rate', 'highest-rate':'highest_rate', 'current-rate':'current_rate'}, inplace=True)
# test_orders['one_on_lowest_rate'] = 1/test_orders.lowest_rate
# test_orders['one_on_highest_rate'] = 1/test_orders.highest_rate
# test_orders['one_on_current_rate'] = 1/test_orders.current_rate
test_orders['y_int'] = [test_orders.liquidity[i] * (math.sqrt(test_orders.highest_rate[i]) - math.sqrt(test_orders.lowest_rate[i])) / (math.sqrt(test_orders.current_rate[i]) - math.sqrt(test_orders.lowest_rate[i])) for i in test_orders.index]
test_orders['fix_trade'] = test_orders.y_int - test_orders.liquidity
test_orders['y_int_minus_fixtrade'] = test_orders.y_int - test_orders.fix_trade
test_orders

,id,liquidity,lowest_rate,highest_rate,current_rate,y_int,fix_trade,y_int_minus_fixtrade
0,0.0,254814732.0,1255.0,2255.0,1256.0,2.177916e+11,2.175368e+11,254814732.0
1,1.0,253827078.0,1254.0,2254.0,2254.0,2.538271e+08,0.000000e+00,253827078.0
2,2.0,252839424.0,1253.0,2253.0,1753.0,4.715233e+08,2.186838e+08,252839424.0
3,3.0,251851770.0,1252.0,2252.0,1253.0,2.151938e+11,2.149420e+11,251851770.0
4,4.0,6913578.0,1004.0,2004.0,1504.0,1.274477e+07,5.831196e+06,6913578.0
...,...,...,...,...,...,...,...,...
251,251.0,250864116.0,1251.0,2251.0,2251.0,2.508641e+08,0.000000e+00,250864116.0
252,252.0,5925924.0,1003.0,2003.0,1004.0,4.912572e+09,4.906646e+09,5925924.0
253,253.0,4938270.0,1002.0,2002.0,2002.0,4.938270e+06,0.000000e+00,4938270.0
254,254.0,3950616.0,1001.0,2001.0,1501.0,7.281537e+06,3.330921e+06,3950616.0


In [4]:
test_orders['y_int'][0]

217791634844.78958

In [5]:
test_orders['fix_trade'][0]

217536820112.78958

To translate this order input (effectively p_a and p_b) into a meaningful order in the simulator we need to transform them
The primary reason for this is that the current_rate (akin to p_marg) starts at the highest_rate (that has been denoted p_a) but the simulator takes the p_marg as being the p_start value in this pair

In [6]:
for i in test_orders.index:
    FastSim.add_strategy('ETH', Decimal(test_orders.y_int[i]), Decimal(test_orders.highest_rate[i]), Decimal(test_orders.lowest_rate[i]),  None, None, None)
    if test_orders.fix_trade[i] > 0:
        # print(i, i*2)
        # Sim.amm_sells('ETH', amt=test_orders.fix_trade[i], execute=True, use_positions=[i*2])['trades']
        FastSim.amm_sells('ETH', amt=test_orders.fix_trade[i], execute=True, use_positions=[i*2], use_positions_matchlevel=[i*2])['trades']
# FastSim.state()['orders'].query('tkn=="ETH"')

In [7]:
254559917268

254559917268

In [8]:
trade_result = FastSim.state()['trades']
trade_result

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,2.175368e+11,ETH,1.292602e+08,USDC,USDCETH,0,1,1683.501684,ETH per USDC
0,0,0,A,AMM sells 217536820113ETH buys 129260158USDC,True,True,None,2.175368e+11,ETH,1.292602e+08,USDC,USDCETH,[0],1,1683.501684,ETH per USDC
0,1.0,1,0,route #4,False,True,None,2.186838e+08,ETH,1.100386e+05,USDC,USDCETH,4,1,1988.071571,ETH per USDC
0,1,1,A,AMM sells 218683846ETH buys 110039USDC,True,True,None,2.186838e+08,ETH,1.100386e+05,USDC,USDCETH,[4],1,1988.071571,ETH per USDC
0,2.0,2,0,route #6,False,True,None,2.149420e+11,ETH,1.279562e+08,USDC,USDCETH,6,1,1680.672269,ETH per USDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,170,170,A,AMM sells 4906646218ETH buys 3460010USDC,True,True,None,4.906646e+09,ETH,3.460010e+06,USDC,USDCETH,[504],1,1418.439716,ETH per USDC
0,171.0,171,0,route #508,False,True,None,3.330921e+06,ETH,1.921987e+03,USDC,USDCETH,508,1,1733.102253,ETH per USDC
0,171,171,A,AMM sells 3330921ETH buys 1922USDC,True,True,None,3.330921e+06,ETH,1.921987e+03,USDC,USDCETH,[508],1,1733.102253,ETH per USDC
0,172.0,172,0,route #510,False,True,None,2.452249e+09,ETH,1.733135e+06,USDC,USDCETH,510,1,1414.427157,ETH per USDC


In [9]:
trade_result['amt1'].values[0]

217536820112.78958

We can see that when we input the order with flipped rates we get p_marg == p_start.

Unfortunately this means that the pricing is flipped relative to the order input

In [10]:
eth_orders = FastSim.state()['orders'].query('tkn=="ETH"')
eth_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,USDCETH,ETH,2.177916e+11,254814732.0,ETH,False,2255.0,1255.0,1256.0,ETH per USDC,1
2,2,USDCETH,ETH,2.538271e+08,253827078.0,ETH,False,2254.0,1254.0,2254.0,ETH per USDC,3
4,4,USDCETH,ETH,4.715233e+08,252839424.0,ETH,False,2253.0,1253.0,1753.0,ETH per USDC,5
6,6,USDCETH,ETH,2.151938e+11,251851770.0,ETH,False,2252.0,1252.0,1253.0,ETH per USDC,7
8,8,USDCETH,ETH,1.274477e+07,6913578.0,ETH,False,2004.0,1004.0,1504.0,ETH per USDC,9
...,...,...,...,...,...,...,...,...,...,...,...,...
502,502,USDCETH,ETH,2.508641e+08,250864116.0,ETH,False,2251.0,1251.0,2251.0,ETH per USDC,503
504,504,USDCETH,ETH,4.912572e+09,5925924.0,ETH,False,2003.0,1003.0,1004.0,ETH per USDC,505
506,506,USDCETH,ETH,4.938270e+06,4938270.0,ETH,False,2002.0,1002.0,2002.0,ETH per USDC,507
508,508,USDCETH,ETH,7.281537e+06,3950616.0,ETH,False,2001.0,1001.0,1501.0,ETH per USDC,509


## Route by Source

#### AMM buys USDC

We can then do a similar thing for the fast router

In [11]:
FastSim.amm_buys('USDC',10000000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,173.0,173,0,route #2,False,False,None,2.538271e+08,ETH,1.509775e+05,USDC,USDCETH,2,1,1548.218083,ETH per USDC
0,173.1,173,1,route #502,False,False,None,2.508641e+08,ETH,1.494934e+05,USDC,USDCETH,502,1,1548.218083,ETH per USDC
0,173.2,173,2,route #496,False,False,None,2.479012e+08,ETH,1.480039e+05,USDC,USDCETH,496,1,1548.218083,ETH per USDC
0,173.3,173,3,route #20,False,False,None,2.449382e+08,ETH,1.465088e+05,USDC,USDCETH,20,1,1548.218083,ETH per USDC
0,173.4,173,4,route #484,False,False,None,2.419752e+08,ETH,1.450081e+05,USDC,USDCETH,484,1,1548.218083,ETH per USDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,173.100,173,100,route #22,False,False,None,1.382716e+07,ETH,9.697309e+03,USDC,USDCETH,22,1,1548.218083,ETH per USDC
0,173.101,173,101,route #392,False,False,None,1.965431e+08,ETH,1.380001e+05,USDC,USDCETH,392,1,1548.218083,ETH per USDC
0,173.102,173,102,route #16,False,False,None,1.086419e+07,ETH,7.636342e+03,USDC,USDCETH,16,1,1548.218083,ETH per USDC
0,173.103,173,103,route #386,False,False,None,3.667345e+06,ETH,2.172790e+03,USDC,USDCETH,386,1,1548.218083,ETH per USDC


## Route by Target

#### AMM sells ETH

In [12]:
FastSim.amm_sells('ETH',15482095361, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,174.0,174,0,route #2,False,False,None,2.538271e+08,ETH,1.509775e+05,USDC,USDCETH,2,1,1547.987616,ETH per USDC
0,174.1,174,1,route #502,False,False,None,2.508641e+08,ETH,1.494934e+05,USDC,USDCETH,502,1,1547.987616,ETH per USDC
0,174.2,174,2,route #496,False,False,None,2.479012e+08,ETH,1.480039e+05,USDC,USDCETH,496,1,1547.987616,ETH per USDC
0,174.3,174,3,route #20,False,False,None,2.449382e+08,ETH,1.465088e+05,USDC,USDCETH,20,1,1547.987616,ETH per USDC
0,174.4,174,4,route #484,False,False,None,2.419752e+08,ETH,1.450081e+05,USDC,USDCETH,484,1,1547.987616,ETH per USDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,174.100,174,100,route #22,False,False,None,1.382716e+07,ETH,9.697309e+03,USDC,USDCETH,22,1,1547.987616,ETH per USDC
0,174.101,174,101,route #392,False,False,None,1.965431e+08,ETH,1.380001e+05,USDC,USDCETH,392,1,1547.987616,ETH per USDC
0,174.102,174,102,route #16,False,False,None,1.086419e+07,ETH,7.636342e+03,USDC,USDCETH,16,1,1547.987616,ETH per USDC
0,174.103,174,103,route #386,False,False,None,3.581873e+06,ETH,2.122000e+03,USDC,USDCETH,386,1,1547.987616,ETH per USDC


In [13]:
def import_results_file(filename, is_by_target=True):
    f = open(filename , "r")
    bm = f.read()
    if is_by_target:
        bm = bm.split('matchByTargetAmount')[1]
    else:
        bm = bm.split('matchByTargetAmount')[0]
    bm = bm.split('\n')
    bm = bm[1:]
    print(len(bm))
    bm = [x.replace('- order ','').replace(": ",",").replace('input = ','').replace(" output = ","") for x in bm]
    bm = [x.split(',') for x in bm]
    df = pd.DataFrame(bm, columns = ['result_id', 'input', 'output'])
    df.fillna('0', inplace=True)
    df = df[(df.input!='0')&(df.output!='0')].copy()
    df = df.astype(int)
    f.close()
    return(df)

In [14]:
df = import_results_file("256 orders with curr rate alternating - results.txt", is_by_target=False)
df

106


,result_id,input,output
0,1,150978,253827078
1,251,149494,250864116
2,248,148004,247901154
3,10,146509,244938192
4,242,145009,241975230
...,...,...,...
99,199,139782,199506108
100,11,9698,13827156
101,196,138001,196543146
102,8,7637,10864194


In [15]:
bysrc = FastSim.state()['trades'].query("id==173").query("subid!='A'")
bysrc.reset_index(inplace=True, drop=True)
bysrc['result_id'] = bysrc.routeix//2
resdf = pd.merge(bysrc, df, how='outer', on='result_id')

resdf['input_delta_perc'] = (resdf.amt1 - resdf.input) / resdf.input
resdf['output_delta_perc'] = (resdf.amt2 - resdf.output) / resdf.output
resdf

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,...,pair,routeix,nroutes,price,p_unit,result_id,input,output,input_delta_perc,output_delta_perc
0,173.0,173,0,route #2,False,False,None,2.538271e+08,ETH,150977.498776,...,USDCETH,2,1,1548.218083,ETH per USDC,1,150978,253827078,1680.218972,-0.999405
1,173.1,173,1,route #502,False,False,None,2.508641e+08,ETH,149493.441950,...,USDCETH,502,1,1548.218083,ETH per USDC,251,149494,250864116,1677.088191,-0.999404
2,173.2,173,2,route #496,False,False,None,2.479012e+08,ETH,148003.880519,...,USDCETH,496,1,1548.218083,ETH per USDC,248,148004,247901154,1673.962528,-0.999403
3,173.3,173,3,route #20,False,False,None,2.449382e+08,ETH,146508.782521,...,USDCETH,20,1,1548.218083,ETH per USDC,10,146509,244938192,1670.830345,-0.999402
4,173.4,173,4,route #484,False,False,None,2.419752e+08,ETH,145008.115739,...,USDCETH,484,1,1548.218083,ETH per USDC,242,145009,241975230,1667.691116,-0.999401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,173.99,173,99,route #398,False,False,None,1.995061e+08,ETH,139781.570435,...,USDCETH,398,1,1548.218083,ETH per USDC,199,139782,199506108,1426.266086,-0.999299
100,173.100,173,100,route #22,False,False,None,1.382716e+07,ETH,9697.309338,...,USDCETH,22,1,1548.218083,ETH per USDC,11,9698,13827156,1424.773974,-0.999299
101,173.101,173,101,route #392,False,False,None,1.965431e+08,ETH,138000.096341,...,USDCETH,392,1,1548.218083,ETH per USDC,196,138001,196543146,1423.215375,-0.999298
102,173.102,173,102,route #16,False,False,None,1.086419e+07,ETH,7636.342377,...,USDCETH,16,1,1548.218083,ETH per USDC,8,7637,10864194,1421.573524,-0.999297


In [16]:
max([abs(x) for x in resdf.output_delta_perc])

0.9994051954583979

In [17]:
resdf.input.sum()

10000000

In [18]:
resdf.amt2.sum()

10000000.000003

In [19]:
resdf.output.sum()

15482095361

In [20]:
resdf.amt1.sum()

15482180832.712845

In [21]:
(resdf.amt1.sum() - resdf.output.sum()) / resdf.output.sum()

5.520681203150008e-06

In [22]:
df = import_results_file("256 orders with curr rate alternating - results.txt", is_by_target=True)
df

105


,result_id,input,output
0,1,253827078,150978
1,251,250864116,149494
2,248,247901154,148004
3,10,244938192,146509
4,242,241975230,145009
...,...,...,...
99,199,199506108,139782
100,11,13827156,9698
101,196,196543146,138001
102,8,10864194,7637


In [23]:
bytrg = FastSim.state()['trades'].query("id==174").query("subid!='A'")
bytrg.reset_index(inplace=True, drop=True)
bytrg['result_id'] = bytrg.routeix//2
trgresdf = pd.merge(bytrg, df, how='outer', on='result_id')

trgresdf['input_delta_perc'] = (trgresdf.amt1 - trgresdf.input) / trgresdf.input
trgresdf['output_delta_perc'] = (trgresdf.amt2 - trgresdf.output) / trgresdf.output
trgresdf

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,...,pair,routeix,nroutes,price,p_unit,result_id,input,output,input_delta_perc,output_delta_perc
0,174.0,174,0,route #2,False,False,None,2.538271e+08,ETH,150977.498776,...,USDCETH,2,1,1547.987616,ETH per USDC,1,253827078,150978,0.000000e+00,-3.319848e-06
1,174.1,174,1,route #502,False,False,None,2.508641e+08,ETH,149493.441950,...,USDCETH,502,1,1547.987616,ETH per USDC,251,250864116,149494,0.000000e+00,-3.732926e-06
2,174.2,174,2,route #496,False,False,None,2.479012e+08,ETH,148003.880519,...,USDCETH,496,1,1547.987616,ETH per USDC,248,247901154,148004,0.000000e+00,-8.072822e-07
3,174.3,174,3,route #20,False,False,None,2.449382e+08,ETH,146508.782521,...,USDCETH,20,1,1547.987616,ETH per USDC,10,244938192,146509,0.000000e+00,-1.484407e-06
4,174.4,174,4,route #484,False,False,None,2.419752e+08,ETH,145008.115739,...,USDCETH,484,1,1547.987616,ETH per USDC,242,241975230,145009,0.000000e+00,-6.097973e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,174.99,174,99,route #398,False,False,None,1.995061e+08,ETH,139781.570435,...,USDCETH,398,1,1547.987616,ETH per USDC,199,199506108,139782,0.000000e+00,-3.073107e-06
100,174.100,174,100,route #22,False,False,None,1.382716e+07,ETH,9697.309338,...,USDCETH,22,1,1547.987616,ETH per USDC,11,13827156,9698,0.000000e+00,-7.121695e-05
101,174.101,174,101,route #392,False,False,None,1.965431e+08,ETH,138000.096341,...,USDCETH,392,1,1547.987616,ETH per USDC,196,196543146,138001,0.000000e+00,-6.548206e-06
102,174.102,174,102,route #16,False,False,None,1.086419e+07,ETH,7636.342377,...,USDCETH,16,1,1547.987616,ETH per USDC,8,10864194,7637,0.000000e+00,-8.611012e-05


In [24]:
max([abs(x) for x in trgresdf.output_delta_perc])

8.611012177557389e-05

In [25]:
trgresdf.input.sum()

15482095361

In [26]:
trgresdf.amt1.sum()

15482095361.000002

In [27]:
trgresdf.output.sum()

10000000

In [28]:
trgresdf.amt2.sum()

9999949.209725

In [29]:
(trgresdf.amt2.sum() - trgresdf.output.sum())/trgresdf.output.sum()

-5.079027500003576e-06